<a href="https://colab.research.google.com/github/lalaisp/pcd/blob/main/pcd2_cuda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
%%writefile difusao.c
#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#define N 2000  // Tamanho da grade
#define T 500 // Número de iterações no tempo
#define D 0.1  // Coeficiente de difusão
#define DELTA_T 0.01
#define DELTA_X 1.0

void diff_eq(double **C, double **C_new) { //diff_eq(double C[N][N], double C_new[N][N]) {
    for (int t = 0; t < T; t++) {
        for (int i = 1; i < N - 1; i++) {
            for (int j = 1; j < N - 1; j++) {
                C_new[i][j] = C[i][j] + D * DELTA_T * (
                    (C[i+1][j] + C[i-1][j] + C[i][j+1] + C[i][j-1] - 4 * C[i][j]) / (DELTA_X * DELTA_X)
                );
            }
        }
        // Atualizar matriz para a próxima iteração
        double difmedio = 0.;
        for (int i = 1; i < N - 1; i++) {
            for (int j = 1; j < N - 1; j++) {
                difmedio += fabs(C_new[i][j] - C[i][j]);
                C[i][j] = C_new[i][j];
            }
        }
        if ((t%100) == 0)
          printf("interacao %d - diferenca=%g\n", t, difmedio/((N-2)*(N-2)));
    }
}

int main() {

    // Concentração inicial
    double **C = (double **)malloc(N * sizeof(double *));
    if (C == NULL) {
      fprintf(stderr, "Memory allocation failed\n");
      return 1;
    }
    for (int i = 0; i < N; i++) {
      C[i] = (double *)malloc(N * sizeof(double));
      if (C[i] == NULL) {
        fprintf(stderr, "Memory allocation failed\n");
        return 1;
      }
    }
    for (int i = 0; i < N; i++) {
      for (int j = 0; j < N; j++) {
        C[i][j] = 0.;
      }
    }

    // Concentração para a próxima iteração
    double **C_new = (double **)malloc(N * sizeof(double *));
    if (C_new == NULL) {
      fprintf(stderr, "Memory allocation failed\n");
      return 1;
    }
    for (int i = 0; i < N; i++) {
      C_new[i] = (double *)malloc(N * sizeof(double));
      if (C_new[i] == NULL) {
        fprintf(stderr, "Memory allocation failed\n");
        return 1;
      }
    }
    for (int i = 0; i < N; i++) {
      for (int j = 0; j < N; j++) {
        C_new[i][j] = 0.;
      }
    }

    // Inicializar uma concentração alta no centro
    C[N/2][N/2] = 1.0;

    // Executar as iterações no tempo para a equação de difusão
    diff_eq(C, C_new);

    // Exibir resultado para verificação
    printf("Concentração final no centro: %f\n", C[N/2][N/2]);
    return 0;
}

Writing difusao.c


In [22]:
!rm difusao.x
!gcc difusao.c -o difusao.x
!time ./difusao.x

rm: cannot remove 'difusao.x': No such file or directory
interacao 0 - diferenca=2.00401e-09
interacao 100 - diferenca=1.23248e-09
interacao 200 - diferenca=7.81794e-10
interacao 300 - diferenca=5.11528e-10
interacao 400 - diferenca=4.21632e-10
Concentração final no centro: 0.216512

real	0m30.293s
user	0m30.040s
sys	0m0.040s


In [23]:
!more /proc/cpuinfo &> processador.txt
!more processador.txt | grep model

model		: 85
model name	: Intel(R) Xeon(R) CPU @ 2.00GHz
model		: 85
model name	: Intel(R) Xeon(R) CPU @ 2.00GHz


In [18]:
%%writefile pcd2_1.cu

#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <cuda_runtime.h>

//#define N 2000 // Tamanho da grade
#define T 500  // Número de iterações
#define D 0.1  // Coeficiente de difusão
#define DELTA_T 0.01 // Passo temporal
#define DELTA_X 1.0  // Passo espacial

// Kernel CUDA para computar a equação
__global__ void diff_eq_kernel(double* C, double* C_new, int N) {
    int i = blockIdx.y * blockDim.y + threadIdx.y + 1;
    int j = blockIdx.x * blockDim.x + threadIdx.x + 1;

    if (i < N - 1 && j < N - 1) {
        C_new[i * N + j] = C[i * N + j] + D * DELTA_T * (
            (C[(i + 1) * N + j] + C[(i - 1) * N + j] + C[i * N + (j + 1)] + C[i * N + (j - 1)] - 4 * C[i * N + j]) /
            (DELTA_X * DELTA_X)
        );
    }
}

// Kernel CUDA para atualizar a matriz e calcular a diferença média
__global__ void update_kernel(double* C, double* C_new, double* difmedio, int N) {

    __shared__ double dif_shared[256]; // depende do número de threads por bloco, vai ser blockDim.x * blockDim.y entao se for 2x2 vai ser 4 threads

    int tid = threadIdx.x + threadIdx.y * blockDim.x;
    int i = blockIdx.y * blockDim.y + threadIdx.y + 1;
    int j = blockIdx.x * blockDim.x + threadIdx.x + 1;

    double local_dif = 0.0;

    if (i < N - 1 && j < N - 1) {
        local_dif = fabs(C_new[i * N + j] - C[i * N + j]);
        C[i * N + j] = C_new[i * N + j];
    }

    dif_shared[tid] = local_dif;
    __syncthreads();

    // Redução paralela para somar difmedio !!
    for (int stride = blockDim.x * blockDim.y / 2; stride > 0; stride /= 2) {
        if (tid < stride) {
            dif_shared[tid] += dif_shared[tid + stride];
        }
        __syncthreads();
    }

    if (tid == 0) {
        //atomicAdd(difmedio, dif_shared[0]); //colab deu erro
        atomicAdd(reinterpret_cast<unsigned long long int*>(difmedio), __double_as_longlong(dif_shared[0]));

    }
}


int main() {
    double *C, *C_new, *d_C, *d_C_new, *d_difmedio;
    //double difmedio = 0.0;
    int N = 2000;

    size_t size = N * N * sizeof(double);

    // Alocação de memória no host
    C = (double*)calloc(N * N, sizeof(double));
    C_new = (double*)calloc(N * N, sizeof(double));
    C[N / 2 * N + N / 2] = 1.0; // Concentração inicial no centro

    // Alocação de memória na GPU
    cudaMalloc(&d_C, size);
    cudaMalloc(&d_C_new, size);
    cudaMalloc(&d_difmedio, sizeof(double));

    cudaMemcpy(d_C, C, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_C_new, C_new, size, cudaMemcpyHostToDevice);

    dim3 threadsPerBlock(16, 16); //defino o numero de threads por bloco AQUI

    dim3 numBlocks((N + threadsPerBlock.x - 1) / threadsPerBlock.x,
                   (N + threadsPerBlock.y - 1) / threadsPerBlock.y);

    double start = clock();

    for (int t = 0; t < T; t++) {
        cudaMemset(d_difmedio, 0, sizeof(double));

        diff_eq_kernel<<<numBlocks, threadsPerBlock>>>(d_C, d_C_new, N);
        update_kernel<<<numBlocks, threadsPerBlock>>>(d_C, d_C_new, d_difmedio, N);

        cudaDeviceSynchronize();

        // Exibe a diferença média a cada 100 iterações igual dos prof
        // if (t % 100 == 0) {
          //   cudaMemcpy(&difmedio, d_difmedio, sizeof(double), cudaMemcpyDeviceToHost);
           //  printf("Iteração %d - Diferença média = %g\n", t, difmedio / ((N - 2) * (N - 2)));
        // }
    }

    cudaMemcpy(C, d_C, size, cudaMemcpyDeviceToHost);

    double end = clock();

    printf("Concentração final: %f\n", C[(N / 2 - 2) * N + (N / 2 - 2)]);
    printf("Tempo: %f\n", (end - start) / CLOCKS_PER_SEC);

    // Liberação de memória
    cudaFree(d_C);
    cudaFree(d_C_new);
    cudaFree(d_difmedio);
    free(C);
    free(C_new);

    return 0;
}


Overwriting pcd2_1.cu


In [19]:
%%shell
nvcc pcd2_1.cu -o pcd2


In [9]:
%%shell
./pcd2
//rodando com 2 threads

Concentração final: 0.002490
Tempo: 1.734228


In [14]:
%%shell
./pcd2
//rodando com 4 threads

Concentração final: 0.002490
Tempo: 0.681926


In [17]:
%%shell
./pcd2
//rodando com 8 threads

Concentração final: 0.002490
Tempo: 0.550121


In [20]:
%%shell
./pcd2


Concentração final: 0.002490
Tempo: 0.567217
